In [1]:
#!/usr/bin/env python3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import astropy.units as u
from sunpy.time import TimeRange
import time
import sys

import datetime as dt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

%matplotlib inline

In [2]:
sys.path.insert(1, '../')
from sk import LofarRaw

In [3]:
filename = '../udpoutput/jupiter-stokesI_0_2020-10-13T17:45:00_19563123779296'
frange = [15,60]
sbs = np.arange(76,320)
obs_mode = 3
time_len_mins = 15.
trange = TimeRange(filename.split('_')[-2], time_len_mins*u.min)
xlabel = "Time from {} {}".format(filename.split('_')[-2].split('T')[0], filename.split('_')[-2].split('T')[1])
ylabel = "Frequency (MHz)"
title = filename.split('/')[2]

Read in the data to raw using the class I made from before. It is essentially just doing np.fromfile() at this point. 
The method for plotting the data will be simple. 

* Normalise the data (in frequency response) split by smaller time chunks
* Fit 2nd order polynomials to the data (split into smaller chunks again)
* Plot the data to see what it looks like and make sure its all ok (beware imshow not plotting entire dataset if large)
* Decimate the data (integrating to downsample the time axis)
* final plot

In [4]:
raw = LofarRaw(fname=filename, sbs=sbs, obs_mode=obs_mode, frange=frange)
flimits = np.array([40,94])
freqlimits = raw.sb_to_f(flimits+sbs[0], obs_mode)
xlims = list(map(dt.datetime.fromisoformat, [trange.start.value, trange.end.value]))
xlims = mdates.date2num(xlims)
raw.data = raw.data[:, flimits[0]:flimits[1]]

Time to read data: 0.000697s



In [5]:
#raw.data = raw.data[::100]
norm_seconds = 30 #the resolution of poly fitting
norm_splits = int((15*60)/norm_seconds)
print(norm_splits)
poly_splits = 1000

30


In [6]:
arrays = np.array_split(raw.data, norm_splits)
#arrays = [(arr/np.quantile(arr, 0.1, axis=0)) for arr in arrays]
for i,arr in enumerate(arrays):
    arr = arr/np.quantile(np.abs(arr), 0.1, axis=0)
    arrays[i] = arr
data = np.vstack(arrays)

In [7]:
#rfi_bands = [0, 1, 6, 7, 25, 51, 52, 53] #dominant rfi bands 0->20 + 31, 32, 38
rfi_bands = [6, 23, 25]
rfi_freqs = raw.sb_to_f(np.array(rfi_bands) + flimits[0] + sbs[0], obs_mode)
print("\nMasked subbands: {}\nMasked frequencies: {}\n".format(np.array(rfi_bands)+flimits[0], rfi_freqs))


Masked subbands: [46 63 65]
Masked frequencies: [23.828125  27.1484375 27.5390625] MHz



In [8]:
data[:, rfi_bands] = np.nan ###removing rfi bands from data(set to nan, will be masked later)

In [9]:
plt.figure(figsize=(12,6))
im = plt.imshow(data.T, aspect='auto', origin='lower',
            vmin=np.nanpercentile(data.T, 5), 
            vmax=np.nanpercentile(data.T, 90),
            extent=[xlims[0], xlims[1], 
                    freqlimits[0].value, freqlimits[1].value])

plt.gca().xaxis_date()
date_format = mdates.DateFormatter('%H:%M:%S')
plt.gca().xaxis.set_major_formatter(date_format)

plt.title("Jupiter auroral emission - Stokes V")
plt.ylabel(ylabel, fontsize=14)
plt.xlabel(xlabel, fontsize=14)

divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

plt.tight_layout()

In [9]:
arrs = np.array_split(data, poly_splits)
for arr in arrs:
    for i in np.arange(arr.shape[1]):
        if np.any(np.isnan(arr[:,i])):
            pass
        else:
            coeffs = np.polyfit(np.linspace(0, len(arr[:,i]), len(arr[:, i])), arr[:, i], 2)
            ynew = np.poly1d(coeffs)
            arr[:, i] = ynew(np.linspace(0, len(arr[:, i]), len(arr[:, i])))
data = np.vstack(arrs)

In [10]:
data = np.ma.masked_invalid(data)

In [11]:
plt.figure(figsize=(12,6))
im = plt.imshow(data.T, aspect='auto', origin='lower',
            vmin=np.nanpercentile(data.T, 5), 
            vmax=np.nanpercentile(data.T, 90),
            extent=[xlims[0], xlims[1], 
                    freqlimits[0].value, freqlimits[1].value])

plt.gca().xaxis_date()
date_format = mdates.DateFormatter('%H:%M:%S')
plt.gca().xaxis.set_major_formatter(date_format)

plt.title("Jupiter auroral emission - Stokes V")
plt.ylabel(ylabel, fontsize=14)
plt.xlabel(xlabel, fontsize=14)

divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

plt.tight_layout()

In [12]:
norms = np.quantile(np.abs(data), 0.1, axis=0)
data = data/norms

In [13]:
plt.figure(figsize=(12,6))
im = plt.imshow(data.T, aspect='auto', origin='lower',
            vmin=np.nanpercentile(data.T.compressed(), 5), 
            vmax=np.nanpercentile(data.T.compressed(), 90),
            extent=[xlims[0], xlims[1], 
                    freqlimits[0].value, freqlimits[1].value])

plt.gca().xaxis_date()
date_format = mdates.DateFormatter('%H:%M:%S')
plt.gca().xaxis.set_major_formatter(date_format)

plt.title("Jupiter auroral emission - Stokes V")
plt.ylabel(ylabel, fontsize=14)
plt.xlabel(xlabel, fontsize=14)

divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

plt.tight_layout()

In [15]:
dec = 32
summed = np.cumsum(data, axis=0)
decimated = summed[dec::dec, :] - summed[:-dec:dec, :]

In [16]:
plt.figure(figsize=(12,6))
im = plt.imshow(decimated.T, aspect='auto', origin='lower',
            vmin=np.nanpercentile(decimated.compressed(), 5), 
            vmax=np.nanpercentile(decimated.compressed(), 95),
            extent=[xlims[0], xlims[1], 
                    freqlimits[0].value, freqlimits[1].value])

plt.gca().xaxis_date()
date_format = mdates.DateFormatter('%H:%M:%S')
plt.gca().xaxis.set_major_formatter(date_format)

plt.title("Jupiter auroral emission - Stokes V")
plt.ylabel(ylabel, fontsize=16)
plt.xlabel(xlabel, fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

plt.tight_layout()

In [18]:
plt.savefig('jupiter_stokesI_data_paperplot.png')